In [ ]:
from context import opencaching, openroute, matrix_utils

def print_response(response: dict):
    matrix_utils.print_matrix(response['distances'], "Distances")
    print('\n')
    matrix_utils.print_matrix(response['durations'], "Durations")

In [ ]:
openroute.api.go_offline()
local_ready = openroute.check_status()
print("[{}] localhost".format(' OK ' if local_ready else 'FAIL'))

openroute.api.go_online()
azure_ready = openroute.check_status()
print("[{}] azure".format(' OK ' if azure_ready else 'FAIL'))

openroute.api.toggle_online(not local_ready)

print('Using: {}'.format('azure' if openroute.api.is_online() else 'localhost'))

In [ ]:
start_coord = (52.518848, 13.399411)

In [ ]:
start_lat, start_lon = start_coord
geocaches = opencaching.parse_results(opencaching.search(start_lat, start_lon, count=500))

locations_latlon = [(cache['latitude'], cache['longitude']) for cache in geocaches]
locations_lonlat = [(cache['longitude'], cache['latitude']) for cache in geocaches]

print(f"Found {len(geocaches)} geocaches near {start_coord}")

In [ ]:
import folium

def create_marker(cache: dict, color: str = 'darkred'):
    return folium.map.Marker(
        location=[cache['latitude'], cache['longitude']],
        popup=f"<strong>{cache['code']}</strong><br>Lat:&nbsp;{cache['latitude']:.3f}<br>Lon:&nbsp;{cache['longitude']:.3f}",
        icon=folium.map.Icon(color=color), tooltip=cache['code']
    )

def fit_locations(map: folium.Map, locations: list):
    sw = min(locations)
    ne = max(locations)
    map.fit_bounds([sw, ne])

m = folium.Map(tiles='Stamen Toner',location=start_coord, zoom_start=11)

In [ ]:
for cache in geocaches[:10]:
    create_marker(cache, 'darkblue').add_to(m)
fit_locations(m, locations_latlon[:10])
m

In [ ]:
matrix_10x10 = openroute.request_matrix({
    "locations": locations_lonlat[:10],
    "metrics": ["distance", "duration"]
}, nocache=True)

In [ ]:
print_response(matrix_10x10)

In [ ]:
import json

with open("matrix10.json", 'w+') as outfile:
    json.dump(matrix_10x10, outfile)

In [ ]:
for cache in geocaches[10:50]:
    create_marker(cache, 'darkgreen').add_to(m)
fit_locations(m, locations_latlon[:50])
m

In [ ]:
matrix_50x50 = openroute.matrix_builder(locations_lonlat[:50], nocache=True)

In [ ]:
print_response(matrix_50x50)

In [ ]:
with open("matrix50.json", 'w+') as outfile:
    json.dump(matrix_50x50, outfile)

In [ ]:
for cache in geocaches[50:500]:
    create_marker(cache).add_to(m)
m

In [ ]:
matrix_500x500 = openroute.matrix_builder(locations_lonlat, increments=20)

In [ ]:
print_response(matrix_500x500)

In [ ]:
with open("matrix500.json", 'w+') as outfile:
    json.dump(matrix_500x500, outfile)